In [1]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime
from IPython.display import clear_output

* количество просмотров перевести в рейтинг
* качество просмотров тоже в рейтинг
у нас нет просмотров по фильмам

In [2]:
df_items = pd.read_csv('lab10data/lab10_items.csv', sep='\t')
for col in ['datetime_availability_start','datetime_availability_stop','datetime_show_start','datetime_show_stop']:
    df_items[col] = df_items[col].apply(lambda t: datetime.strptime(t[:16], '%Y-%m-%dT%H:%M') if str(t) != 'nan' else np.nan)
df_items.set_index('item_id',drop=False,inplace=True,verify_integrity=True)
df_items.sort_index(inplace=True)

In [3]:
import json

In [87]:
df_items.head

NameError: name 'df_items' is not defined

In [9]:
df_paid = pd.DataFrame(df_items[df_items['content_type'] == 1])

In [10]:
df_paid.index = range(len(df_paid))

In [11]:
df_paid.head(n=3)

,item_id,channel_id,datetime_availability_start,datetime_availability_stop,datetime_show_start,datetime_show_stop,content_type,title,year,genres,region_id
0,326,NaN,1970-01-01,2099-12-31 21:00:00,NaT,NaT,1,византия,2012.0,"Ужасы,Триллеры,Драмы,Фантастика,Зарубежные",NaN
1,336,NaN,1970-01-01,2099-12-31 21:00:00,NaT,NaT,1,"девственники, берегитесь!",2012.0,"Ужасы,Комедии,Фантастика,Зарубежные",NaN
2,357,NaN,1970-01-01,2099-12-31 21:00:00,NaT,NaT,1,8 первых свиданий,2012.0,"Комедии,Мелодрамы,Наши",NaN


In [12]:
# genres
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [13]:
genres = [[cat.strip() for cat in ls.strip().split(',')] if str(ls) != 'nan' and len(ls.strip()) > 0 else [] for ls in df_items['genres']]
mlb.fit(genres)
genres_tup = [k for k in enumerate(mlb.classes_)]
genres = mlb.transform(genres)

In [14]:
mt = np.matrix(genres)

In [16]:
df_genres = pd.concat([
    df_items[['item_id','content_type']],
    pd.DataFrame(mt, index=df_items.index, dtype='int8')
], axis=1)

In [17]:
# clean up
del genres
del mt
del df_items

In [18]:
df_genres.head(n=2)

,item_id,content_type,0,1,2,3,4,5,6,7,...,74,75,76,77,78,79,80,81,82,83
item_id,,,,,,,,,,,,,,,,,,,,,
326,326,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336,336,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_train = pd.read_csv('lab10data/lab10_train.csv',sep=',')

In [20]:
df_train = df_train.merge(df_genres, how='left', on='item_id')

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [21]:
df_paying_profiles = df_train[df_train['purchase'] == 1].groupby('user_id').sum()
# +3%?
# df_paying_profiles.loc[df_paying_profiles['purchase'] <= 2,[x for x in range(mt.shape[1])]] = 0
len(df_paying_profiles)

1675

In [22]:
df_nonpaying_profiles = df_train.groupby('user_id').sum()
df_nonpaying_profiles = df_nonpaying_profiles[df_nonpaying_profiles['purchase'] == 0]
df_nonpaying_profiles[[x for x in range(len(df_genres.columns)-2)]] = 0
len(df_nonpaying_profiles)

266

In [23]:
df_all_profiles = pd.concat([df_paying_profiles, df_nonpaying_profiles], axis=0)
df_all_profiles.drop(['content_type','item_id'], axis=1, inplace=True)
df_all_profiles['user_id'] = df_all_profiles.index
df_all_profiles.rename(columns={'purchase':'purchase_count'}, inplace=True)
df_all_profiles.head(n=3)

,purchase_count,0,1,2,3,4,5,6,7,8,...,75,76,77,78,79,80,81,82,83,user_id
user_id,,,,,,,,,,,,,,,,,,,,,
1654,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1654
510087,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,510087
517612,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,517612


In [24]:
del df_paying_profiles
del df_nonpaying_profiles
del df_train

## use all

In [26]:
df_test = pd.read_csv('lab10data/lab10_test.csv',sep=',')

In [27]:
df_test_items = df_test.groupby('item_id').count()

In [28]:
df_test_items['item_id'] = df_test_items.index

In [29]:
df_test_items.index = range(len(df_test_items))

In [30]:
df_test_items.drop(['purchase','user_id'],axis=1,inplace=True)

In [35]:
df_test_items[:3]

,item_id
0,326
1,336
2,357


In [42]:
df_mt_test = df_test_items.merge(df_genres, how='left', on='item_id')

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [43]:
df_mt_test.drop(['item_id'], axis=1, inplace=True)

In [44]:
mt_test = df_mt_test.values[:,2:]

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
cossim = cosine_similarity(df_all_profiles.values[:,3:], mt_test)

In [51]:
import pickle

In [52]:
pickle.dump(cossim, open('cossim.pcl','wb'))

In [53]:
pickle.dump(df_test_items, open('df_test_items.pcl','wb'))

In [54]:
pickle.dump(df_all_profiles, open('df_all_profiles.pcl','wb'))

In [59]:
pickle.dump(df_genres, open('df_genres.pcl','wb'))

In [55]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime
from IPython.display import clear_output

In [56]:
import pickle

In [57]:
def prepare_XY(df):
    df_genres = pickle.load(open('df_genres.pcl','rb'))
    df_all_profiles = pickle.load(open('df_all_profiles.pcl','rb'))
    df = df.merge(df_genres, how='left', on='item_id')
    df = df.merge(df_all_profiles, how='left', on='user_id')
    Y = np.array([y for y in df['purchase']])
    df.drop(['purchase','user_id','item_id','content_type'],axis=1,inplace=True)
    df = df.astype('int16')
    del df_genres
    del df_all_profiles
    return (df.values, Y)

In [ ]:
#mt_test = df_test_items.merge(df_genres, how='left', on='item_id').values[:,2:]

In [67]:
df_train = pd.read_csv('lab10data/lab10_train.csv',sep=',')
X_train, Y_train = prepare_XY(df_train)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  after removing the cwd from sys.path.
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: 'user_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """


In [62]:
del df_train

In [68]:
df_test = pd.read_csv('lab10data/lab10_test.csv',sep=',')
X_test, Y_test = prepare_XY(df_test)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  after removing the cwd from sys.path.
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: 'user_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """


In [69]:
del df_test

In [140]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=25)
skf.get_n_splits(X_train, Y_train)
train_index, test_index = next(skf.split(X_train, Y_train))

In [141]:
xtr = X_train[train_index,:]
xts = X_train[test_index,:]
ytr = Y_train[train_index]
yts = Y_train[test_index]

## Test dataset

In [63]:
def submit():
    df_test = pd.read_csv('lab10data/lab10_test.csv',sep=',')
    X_test, Y_test = prepare_XY(df_test)

    probas = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else model.predict(X_test)
    df_test['purchase'] = probas
    df_test.sort_values(['user_id','item_id'], inplace=True)
    df_test.to_csv('lab10s.csv',index=False)

    del df_test
    del X_test
    del Y_test
    del probas

In [70]:
Y_train = np.array([Y_train]).reshape(-1,1)

In [71]:
Y_test = np.array([Y_test]).reshape(-1,1)

In [72]:
Y_train = Y_train.reshape(-1,1)

In [73]:
Y_test = Y_test.reshape(-1,1)

In [109]:
import keras

In [117]:
from sklearn.metrics import roc_auc_score, mean_squared_error

In [143]:
class SaveModel(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0:
            model.save('nn_epoch_' + str(epoch))

In [142]:
class CountROCAUC(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 5 == 0:
            test_predicted_labels = self.model.predict_proba(xts, batch_size=16384)
            print('roc_auc',epoch, roc_auc_score(yts,test_predicted_labels))

In [148]:
test_predicted_labels = model.predict_proba(xts, batch_size=16384)
print('roc_auc',1, roc_auc_score(yts,test_predicted_labels))

roc_auc 1 0.9217205874343255


/opt/anaconda/envs/bd9/lib/python3.6/site-packages/keras/engine/sequential.py:252: UserWarning: Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).
  warnings.warn('Network returning invalid probability values. '


In [74]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [144]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
neurons = 20

model = Sequential()
# len inputs, 'neuron' neurons in 1 (or 2) hidden layer, with relu activation and dropout
model.add(Dense(neurons, init='uniform', input_shape=(X_train.shape[1],))) 
model.add(Activation('selu'))
model.add(Dropout(0.1))
#one more hidden
model.add(Dense(neurons)) 
model.add(Activation('relu'))
model.add(Dropout(0.1))

#one more hidden
model.add(Dense(3)) 
model.add(Activation('relu'))

# 1 output, linear activation
model.add(Dense(1, init='uniform'))
model.add(Activation('linear'))

model.compile(loss='binary_crossentropy', optimizer='adam')

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, input_shape=(169,), kernel_initializer="uniform")`
  import sys
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="uniform")`


In [ ]:
#model.fit(X_train, Y_train, epochs=100, batch_size=200000)
countROC = CountROCAUC()
saveModel = SaveModel()
model.fit(xtr, ytr,epochs=2000, initial_epoch=1000, batch_size=200000,validation_data=(xts, yts), callbacks=[countROC,saveModel],class_weight='balance')

Train on 4831318 samples, validate on 201306 samples
Epoch 1001/2000
4831318/4831318 [==============================] - 20s 4us/step - loss: 0.0107 - val_loss: 0.1827


/opt/anaconda/envs/bd9/lib/python3.6/site-packages/keras/engine/sequential.py:252: UserWarning: Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).
  warnings.warn('Network returning invalid probability values. '


roc_auc 1000 0.929722540914418
Epoch 1002/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0106 - val_loss: 0.1832
Epoch 1003/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0106 - val_loss: 0.1834
Epoch 1004/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0106 - val_loss: 0.1833
Epoch 1005/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0106 - val_loss: 0.1828
Epoch 1006/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0106 - val_loss: 0.1833
roc_auc 1005 0.9253396053643487
Epoch 1007/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0106 - val_loss: 0.1832
Epoch 1008/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0105 - val_loss: 0.1835
Epoch 1009/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0106 - val_loss: 0.1836
Epoch 1010/2000
4831318/4

roc_auc 1075 0.924453797449168
Epoch 1077/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0106 - val_loss: 0.1847
Epoch 1078/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0111 - val_loss: 0.0423
Epoch 1079/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0113 - val_loss: 0.0257
Epoch 1080/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0111 - val_loss: 0.1844
Epoch 1081/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0109 - val_loss: 0.1846
roc_auc 1080 0.9225822155138668
Epoch 1082/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0108 - val_loss: 0.1838
Epoch 1083/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0108 - val_loss: 0.1839
Epoch 1084/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0107 - val_loss: 0.1842
Epoch 1085/2000
4831318/4

Epoch 1145/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0105 - val_loss: 0.1850
Epoch 1146/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0105 - val_loss: 0.1847
roc_auc 1145 0.9244134806348795
Epoch 1147/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0105 - val_loss: 0.1850
Epoch 1148/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0105 - val_loss: 0.1845
Epoch 1149/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0105 - val_loss: 0.1843
Epoch 1150/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0105 - val_loss: 0.1854
Epoch 1151/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0105 - val_loss: 0.1858
roc_auc 1150 0.9234430119665522
Epoch 1152/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0105 - val_loss: 0.1855
Epoch 1153/2000
4831318/

...

4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0102 - val_loss: 0.1872
Epoch 1758/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0104 - val_loss: 0.1883
Epoch 1759/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0103 - val_loss: 0.1883
Epoch 1760/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0103 - val_loss: 0.1887
Epoch 1761/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0102 - val_loss: 0.1892
roc_auc 1760 0.9245408961221387
Epoch 1762/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0103 - val_loss: 0.1891
Epoch 1763/2000
4831318/4831318 [==============================] - 14s 3us/step - loss: 0.0102 - val_loss: 0.1890
Epoch 1764/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0103 - val_loss: 0.1886
Epoch 1765/2000
4831318/4831318 [==============================] - 15s 3

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0103 - val_loss: 0.1898
Epoch 1856/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0102 - val_loss: 0.1915
roc_auc 1855 0.9223592711635905
Epoch 1857/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0103 - val_loss: 0.1903
Epoch 1858/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0103 - val_loss: 0.1901
Epoch 1859/2000
4831318/4831318 [==============================] - 17s 3us/step - loss: 0.0103 - val_loss: 0.1894
Epoch 1860/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0102 - val_loss: 0.1898
Epoch 1861/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0103 - val_loss: 0.1907
roc_auc 1860 0.9222805457199224
Epoch 1862/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0103 - val_loss: 0.1903
Epoch 1863/2000
4831318/4831318 [=======

4831318/4831318 [==============================] - 17s 4us/step - loss: 0.0103 - val_loss: 0.1914
Epoch 1924/2000
4831318/4831318 [==============================] - 18s 4us/step - loss: 0.0103 - val_loss: 0.1914
Epoch 1925/2000
4831318/4831318 [==============================] - 20s 4us/step - loss: 0.0103 - val_loss: 0.1879
Epoch 1926/2000
4831318/4831318 [==============================] - 22s 5us/step - loss: 0.0103 - val_loss: 0.1896
roc_auc 1925 0.9230058382597636
Epoch 1927/2000
4831318/4831318 [==============================] - 18s 4us/step - loss: 0.0103 - val_loss: 0.1911
Epoch 1928/2000
4831318/4831318 [==============================] - 18s 4us/step - loss: 0.0103 - val_loss: 0.1909
Epoch 1929/2000
4831318/4831318 [==============================] - 15s 3us/step - loss: 0.0103 - val_loss: 0.1898
Epoch 1930/2000
4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0103 - val_loss: 0.1901
Epoch 1931/2000
4831318/4831318 [==============================] - 20s 4

4831318/4831318 [==============================] - 16s 3us/step - loss: 0.0102 - val_loss: 0.1940
roc_auc 1990 0.9231893999519456
Epoch 1992/2000
1000000/4831318 [=====>........................] - ETA: 12s - loss: 0.0104

In [ ]:
submit()